<a href="https://colab.research.google.com/github/Norberto27/Norberto27/blob/main/NaiveBayes_Ver2_NorbertoMartearena.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **VERSION 2**

CON AJUSTE DE HIPERPARAMETROS

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [ ]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [ ]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
# tfidfvect = TfidfVectorizer()
tfidfvect = TfidfVectorizer(max_df=0.8, min_df=2, ngram_range=(1, 2), sublinear_tf=True)

In [ ]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [ ]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
# X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término
X_train = tfidfvect.fit_transform(newsgroups_train.data)
X_test = tfidfvect.transform(newsgroups_test.data)

In [ ]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 230758)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 230758


In [ ]:
# una vez ajustado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

44991

In [ ]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [ ]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [ ]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## **Seleccionar 5 documentos al azar**

In [ ]:
import random
# Seleccionamos 5 documentos al azar del conjunto de entrenamiento
import random

# Fijar una semilla aleatoria
random.seed(27)

# Seleccionar 5 párrafos al azar
random_docs = random.sample(newsgroups_train.data, 5)



# **Medir la similaridad de esos 5 documentos con el resto**

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Calcular la similaridad de los documentos seleccionados con todos los documentos del conjunto
for i, doc in enumerate(random_docs):
    doc_vect = tfidfvect.transform([doc])  # Vectoriza el documento individual
    cossim = cosine_similarity(doc_vect, X_train)[0]  # Similaridad con todos los documentos de train
    mostsim = np.argsort(cossim)[::-1][1:6]  # 5 documentos más similares, excluyendo el mismo documento

    # Muestra la clase del documento seleccionado
    print(f'Documento {i} pertenece a la clase: {newsgroups_train.target_names[y_train[newsgroups_train.data.index(doc)]]}')

    # Muestra los 5 documentos más similares
    print(f'Los 5 documentos más similares son:')
    for idx in mostsim:
        print(f'Clase: {newsgroups_train.target_names[y_train[idx]]}')



Documento 0 pertenece a la clase: soc.religion.christian
Los 5 documentos más similares son:
Clase: rec.motorcycles
Clase: soc.religion.christian
Clase: talk.politics.misc
Clase: soc.religion.christian
Clase: alt.atheism
Documento 1 pertenece a la clase: rec.autos
Los 5 documentos más similares son:
Clase: comp.windows.x
Clase: comp.sys.ibm.pc.hardware
Clase: rec.motorcycles
Clase: comp.graphics
Clase: soc.religion.christian
Documento 2 pertenece a la clase: rec.sport.hockey
Los 5 documentos más similares son:
Clase: rec.sport.hockey
Clase: rec.sport.baseball
Clase: rec.sport.hockey
Clase: rec.sport.hockey
Clase: rec.sport.hockey
Documento 3 pertenece a la clase: rec.motorcycles
Los 5 documentos más similares son:
Clase: rec.motorcycles
Clase: misc.forsale
Clase: misc.forsale
Clase: misc.forsale
Clase: misc.forsale
Documento 4 pertenece a la clase: comp.os.ms-windows.misc
Los 5 documentos más similares son:
Clase: sci.med
Clase: comp.graphics
Clase: comp.os.ms-windows.misc
Clase: comp.

# **Entrenar modelos Naïve Bayes**

In [ ]:
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# Entrenamos el modelo Naive Bayes Multinomial
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Vectorizamos los datos de prueba y predecimos
X_test = tfidfvect.transform(newsgroups_test.data)
y_pred = clf.predict(X_test)

# Calculamos el F1-score con promediado macro
print(f'F1-score (macro): {f1_score(newsgroups_test.target, y_pred, average="macro")}')


F1-score (macro): 0.5745979561373709


# **Transponer la matriz y estudiar la similaridad entre palabras**

In [ ]:
# Transponemos la matriz documento-término
X_train_transposed = X_train.T

# Elegimos manualmente 5 palabras para estudiar su similaridad
palabras = ['car', 'computer', 'government', 'money', 'health']

for palabra in palabras:
    idx = tfidfvect.vocabulary_[palabra]
    palabra_vect = X_train_transposed[idx]  # Vectoriza la palabra individualmente
    similar_words = cosine_similarity(palabra_vect, X_train_transposed)[0]  # Similaridad con otras palabras
    most_similar_idx = np.argsort(similar_words)[::-1][1:6]  # Las 5 palabras más similares
    print(f'Palabra: {palabra}')
    for idx in most_similar_idx:
        print(f'Palabra similar: {idx2word[idx]}')


Palabra: car
Palabra similar: the car
Palabra similar: car and
Palabra similar: car is
Palabra similar: car in
Palabra similar: my car
Palabra: computer
Palabra similar: the computer
Palabra similar: my computer
Palabra similar: computer science
Palabra similar: your computer
Palabra similar: computer graphics
Palabra: government
Palabra similar: the government
Palabra similar: government is
Palabra similar: of government
Palabra similar: government has
Palabra similar: government to
Palabra: money
Palabra similar: the money
Palabra similar: money for
Palabra similar: of money
Palabra similar: money to
Palabra similar: money and
Palabra: health
Palabra similar: health care
Palabra similar: health insurance
Palabra similar: of health
Palabra similar: public health
Palabra similar: the health
